<a href="https://colab.research.google.com/github/AromaR/685_Project/blob/main/POMO_Probing_refactored_bert_tacred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")

Found device: Tesla P4, n_gpu: 1


In [3]:
import pandas as pd
import numpy as np
!pip install transformers
!pip install tqdm
from tqdm import tqdm

     |████████████████████████████████| 1.4MB 6.7MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4d1af6525c21ed6952bfed846ced12c279b8926029e09208af6f73d9e3378186
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("mrm8488/spanbert-large-finetuned-tacred")

# model = AutoModel.from_pretrained("mrm8488/spanbert-large-finetuned-tacred")

In [5]:
train_data = []
with open("/content/drive/My Drive/POMO/emb_04_train", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        train_data.append([[[temp]],int(y[1])])



In [6]:
with open("/content/drive/My Drive/POMO/emb_05_train", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        train_data.append([[[temp]],int(y[1])])


In [7]:
print(len(train_data))

19872


In [8]:
test_data = []
with open("/content/drive/My Drive/POMO/emb_04_test", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        test_data.append([[[temp]],int(y[1])])


In [9]:
print(len(test_data))

10226


In [10]:
train = []
test = []
valid = []
trainans = []
testans = []
validans = []
for i in train_data:
  train.append(i[0])
  trainans.append(i[1])
for i in test_data:
  test.append(i[0])
  testans.append(i[1])
valid = train[15000:]
validans = trainans[15000:]
train = train[0:15000]
trainans = trainans[0:15000]


In [11]:
# dataset = pd.read_csv("/content/drive/My Drive/POMO/train.pm",sep='\\t')
# dataarr = []
# ansarr = []
# for i in range(15000,20000):
#   dataarr.append(dataset.iloc[i,0])
#   ansarr.append(0)
#   dataarr.append(dataset.iloc[i,3])
#   ansarr.append(1)
# torch.set_printoptions(profile="full")


In [12]:
# with open("/content/drive/My Drive/POMO/emb_04_train", "w") as writer:
#   for i in tqdm(range(0, len(dataarr), 128)):
#     batch = dataarr[i:i+128]
#     ans = ansarr[i:i+128]
#     inputs = tokenizer.batch_encode_plus(batch,
#                 add_special_tokens=True,
#                 truncation=True,
#                 padding=True,
#                 return_tensors="pt",
#             )
#     for j in range(0,128):
#       try:
#         writer.write("|")
#         writer.write(str(model(torch.tensor(inputs['input_ids'][j]).unsqueeze(0))[0][0][0]))
#         writer.write("$")
#         writer.write(str(ans[j]))
#       except:
#         continue
# torch.set_printoptions(profile="default")
    


In [13]:
dataset = pd.read_csv("/content/drive/My Drive/POMO/test.pm",sep='\\t')
dataarr_test = []
# ansarr_test = []
# print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
for i in range(0,len(dataset)):
  dataarr_test.append(dataset.iloc[i,0])
#   ansarr_test.append(0)
  dataarr_test.append(dataset.iloc[i,3])
#   ansarr_test.append(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [14]:
# dataset = pd.read_csv("/content/drive/My Drive/POMO/valid.pm",sep='\\t')
# dataarr_valid = []
# ansarr_valid = []
# print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
# for i in range(0,len(dataset)):
#   dataarr_valid.append(dataset.iloc[i,0])
#   ansarr_valid.append(0)
#   dataarr_valid.append(dataset.iloc[i,3])
#   ansarr_valid.append(1)

In [15]:
# print(len(dataarr))
# print(dataarr[0])
# print(ansarr[0])
# print(dataarr[1])
# print(ansarr[1])
# h = []
# for j in ansarr:
#   h.append(torch.LongTensor([j]).unsqueeze(0))
# ansarr1 = torch.stack((h))
# print(ansarr1.shape)

In [16]:
# print(len(dataarr_test))
# print(dataarr_test[0])
# print(ansarr_test[0])
# print(dataarr_test[1])
# print(ansarr_test[1])
# ansarr1_test = torch.from_numpy(np.array(ansarr_test))

In [17]:
# print(len(dataarr_valid))
# print(dataarr_valid[0])
# print(ansarr_valid[0])
# print(dataarr_valid[1])
# print(ansarr_valid[1])
# ansarr1_valid = torch.from_numpy(np.array(ansarr_valid))

Creation of data

Conversion to hugging faces BERT-Tacred fine-tuned representation

In [18]:
class POMO_Dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, _dataset):
        self.dataset = _dataset

    def __getitem__(self, index):
        example= self.dataset[0][index]
        target = self.dataset[1][index]
        target.squeeze(0)
        return np.array(example), target

    def __len__(self):
        return len(self.dataset)

In [19]:
# emb_data = tokenizer.batch_encode_plus(dataarr_valid)
# k = [torch.LongTensor(x) for x in emb_data['input_ids']]
# max_len = max([len(x) for x in emb_data['input_ids']])
# data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
# data = torch.stack(data)
# valid = []
# for i in range(0,len(data)):
#   valid.append([data[i],ansarr1_valid[i]])


In [20]:
# emb_data = tokenizer.batch_encode_plus(dataarr)
# k = [torch.LongTensor(x) for x in emb_data['input_ids']]
# max_len = max([len(x) for x in emb_data['input_ids']])
# data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
# data = torch.stack(data)
# train = []



Get Bert embeddings

In [21]:
# input_ids = torch.tensor(tokenizer.encode("Hello, my name is Aroma")).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids)
# last_hidden_states = outputs[0]

In [22]:
# print(len(last_hidden_states[0][0])) #CLS baby!

In [23]:
# ansarr.unsqueeze(0).unsqueeze(0)

In [24]:
# a = []
# for i in data:
#   a.append(i.unsqueeze(0).unsqueeze(0))

In [25]:
# a = torch.stack(a)

In [26]:
# print(a.shape)

In [27]:
# train = []
# train = [a,ansarr1]

In [28]:
# print(data)

In [29]:
# print((train[0][1]))

In [30]:

# test_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(test),
#                                            batch_size=4,
#                                            shuffle=True)
# valid_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(valid),
#                                            batch_size=4,
#                                            shuffle=True)
# train_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(train),
#                                            batch_size=4,
#                                            shuffle=True)


  
  


In [31]:
# for batch_images, targets in train_loader:
#     print(batch_images.shape)
#     print(targets)
#     break

In [32]:
metrics = [] #we will add mean accuracy on task, control task 1 and control task 2 here



Probing task level 1 : Classifier - binary, Aroma's CNN

In [33]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 2, 1,)
#         self.fc1 = nn.Linear(801, 120)
#         self.fc2 = nn.Linear(120, 1)

#     def forward(self, x):
#         x = (F.relu(self.conv1(x.long()))).long()
#         x = F.relu(self.fc1(x)).long()
#         x = (self.fc2(x)).long()
#         return x


# net = Net()
    
   

In [34]:
# import torch.optim as optim

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [35]:
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs.long())
#         print(labels)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

# print('Finished Training')

In [36]:
!pip install keras

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
model = Sequential()
model.add(Conv2D(64, kernel_size=1, activation='relu', input_shape=(1,1,1024)))
model.add(Conv2D(32, kernel_size=1, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
train.extend(valid)
trainans.extend(validans)

In [40]:
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))

Epoch 1/30
621/621 [==============================] - 35s 56ms/step - loss: 0.6458 - accuracy: 0.6225 - val_loss: 0.6257 - val_accuracy: 0.6411
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6262 - accuracy: 0.6428 - val_loss: 0.6305 - val_accuracy: 0.6161
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6087 - accuracy: 0.6606 - val_loss: 0.5866 - val_accuracy: 0.6859
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5945 - accuracy: 0.6715 - val_loss: 0.5924 - val_accuracy: 0.6714
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5822 - accuracy: 0.6865 - val_loss: 0.5583 - val_accuracy: 0.7091
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5771 - accuracy: 0.6906 - val_loss: 0.5604 - val_accuracy: 0.7075
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5626 - accuracy: 0.7020 - val_loss: 0.5446 - val_accuracy: 0.7170
Epoc

Control Task: Let us see if our model is too complex and makes sense of nonsense

In [41]:
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))

Epoch 1/30
621/621 [==============================] - 34s 55ms/step - loss: 0.7070 - accuracy: 0.4968 - val_loss: 0.6946 - val_accuracy: 0.4974
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6941 - accuracy: 0.5048 - val_loss: 0.6946 - val_accuracy: 0.4933
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6935 - accuracy: 0.5021 - val_loss: 0.6944 - val_accuracy: 0.4944
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6928 - accuracy: 0.5036 - val_loss: 0.6946 - val_accuracy: 0.4950
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6922 - accuracy: 0.5056 - val_loss: 0.6944 - val_accuracy: 0.4956
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6922 - accuracy: 0.5063 - val_loss: 0.6951 - val_accuracy: 0.4962
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6921 - accuracy: 0.5041 - val_loss: 0.6942 - val_accuracy: 0.4936
Epoc

This is a prediction rate that is about the same as a random rate. Let us see if the information learned by the model is by virtue of the language embeddings or not

In [42]:
cont_train = []
for i in train:
  cont_train.append([[np.add(np.array(i[0][0]),np.random.uniform(low=-1, high=1, size=len(i[0][0]) )).tolist()]])
cont_test = []
for i in test:
  cont_test.append([[np.add(np.array(i[0][0]),np.random.uniform(low=-1, high=1, size=len(i[0][0]) )).tolist()]])

In [43]:
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))

Epoch 1/30
621/621 [==============================] - 34s 55ms/step - loss: 0.6887 - accuracy: 0.5628 - val_loss: 0.6716 - val_accuracy: 0.5745
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6386 - accuracy: 0.6262 - val_loss: 0.6806 - val_accuracy: 0.5728
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5903 - accuracy: 0.6742 - val_loss: 0.7149 - val_accuracy: 0.5605
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5118 - accuracy: 0.7367 - val_loss: 0.7924 - val_accuracy: 0.5501
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.4227 - accuracy: 0.7968 - val_loss: 0.9288 - val_accuracy: 0.5446
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.3452 - accuracy: 0.8407 - val_loss: 1.1374 - val_accuracy: 0.5412
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.2787 - accuracy: 0.8778 - val_loss: 1.3241 - val_accuracy: 0.5364
Epoc

In [44]:
metrics.append((a,b,c,model))

As we can see the randomized addition to the embeddings has resulted in worse accuracy. Let us now experiment with a different version of the model

In [45]:
from keras.models import Sequential

from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

model.add(Conv2D(128, kernel_size=1, activation='relu', input_shape=(1,1,1024)))

model.add(Conv2D(64, kernel_size=1, activation='relu'))
model.add(Conv2D(32, kernel_size=1, activation='relu'))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))




Epoch 1/30
621/621 [==============================] - 34s 55ms/step - loss: 0.6455 - accuracy: 0.6242 - val_loss: 0.6273 - val_accuracy: 0.6438
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6226 - accuracy: 0.6447 - val_loss: 0.5975 - val_accuracy: 0.6727
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6155 - accuracy: 0.6527 - val_loss: 0.6237 - val_accuracy: 0.6575
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6004 - accuracy: 0.6672 - val_loss: 0.6254 - val_accuracy: 0.6507
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5860 - accuracy: 0.6794 - val_loss: 0.5642 - val_accuracy: 0.7075
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5771 - accuracy: 0.6878 - val_loss: 0.5768 - val_accuracy: 0.6801
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5607 - accuracy: 0.7058 - val_loss: 0.5600 - val_accuracy: 0.7091
Epoc

In [46]:
a,b,c,d,e,f = [],[],[],[],[],[]
for i in train:
  a.append([i[0][0]])
for j in test:
  b.append([j[0][0]])
for l in cont_train:
  d.append([l[0][0]])
for n in cont_test:
  f.append([n[0][0]])


In [47]:
print(len(a[0][0]))

1024


In [48]:
train = a
test = b
cont_train = d
cont_test = f

In [49]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,1024)))
model.add(Dropout(0.50))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))


Epoch 1/30
621/621 [==============================] - 33s 54ms/step - loss: 0.6761 - accuracy: 0.5627 - val_loss: 0.6588 - val_accuracy: 0.6288
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6721 - accuracy: 0.5702 - val_loss: 0.6520 - val_accuracy: 0.6316
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6692 - accuracy: 0.5738 - val_loss: 0.6449 - val_accuracy: 0.6390
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6655 - accuracy: 0.5798 - val_loss: 0.6484 - val_accuracy: 0.6334
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6624 - accuracy: 0.5866 - val_loss: 0.6465 - val_accuracy: 0.6327
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6578 - accuracy: 0.5990 - val_loss: 0.6353 - val_accuracy: 0.6416
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6586 - accuracy: 0.5936 - val_loss: 0.6267 - val_accuracy: 0.6454
Epoc

In [50]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,1024)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))

Epoch 1/30
621/621 [==============================] - 35s 56ms/step - loss: 0.6409 - accuracy: 0.6267 - val_loss: 0.6603 - val_accuracy: 0.6148
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6223 - accuracy: 0.6497 - val_loss: 0.6067 - val_accuracy: 0.6593
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6098 - accuracy: 0.6583 - val_loss: 0.5898 - val_accuracy: 0.6851
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6015 - accuracy: 0.6699 - val_loss: 0.5799 - val_accuracy: 0.6951
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5898 - accuracy: 0.6775 - val_loss: 0.5723 - val_accuracy: 0.6962
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5790 - accuracy: 0.6920 - val_loss: 0.5722 - val_accuracy: 0.6989
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5735 - accuracy: 0.6947 - val_loss: 0.5891 - val_accuracy: 0.6650
Epoc

In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,1024)))
model.add(Conv1D(filters=16,kernel_size=2, padding='same', activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))

Epoch 1/30
621/621 [==============================] - 34s 56ms/step - loss: 0.6487 - accuracy: 0.6159 - val_loss: 0.6271 - val_accuracy: 0.6413
Epoch 2/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6286 - accuracy: 0.6397 - val_loss: 0.6098 - val_accuracy: 0.6633
Epoch 3/30
621/621 [==============================] - 2s 3ms/step - loss: 0.6148 - accuracy: 0.6530 - val_loss: 0.6004 - val_accuracy: 0.6655
Epoch 4/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5974 - accuracy: 0.6726 - val_loss: 0.5773 - val_accuracy: 0.6935
Epoch 5/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5848 - accuracy: 0.6813 - val_loss: 0.5675 - val_accuracy: 0.6989
Epoch 6/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5754 - accuracy: 0.6931 - val_loss: 0.5638 - val_accuracy: 0.7020
Epoch 7/30
621/621 [==============================] - 2s 3ms/step - loss: 0.5603 - accuracy: 0.7052 - val_loss: 0.5586 - val_accuracy: 0.7053
Epoc

In [ ]:
!pip install keras-tuner


In [ ]:
import kerastuner as kt
import keras
!pip install tensorflow
import tensorflow as tf

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))


In [ ]:

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0001)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))

In [ ]:

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.01)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
preds = model.predict(test)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c,model))
almostright = []
totallywrong = []
m = 0
for i,j in zip(preds,testans):
  if (i>0.4 and i<0.5 and j==1) or (i<0.6 and i>0.5 and j==0):
    almostright.append((m,i))
  if (i<0.20 and j==1) or (i>0.80 and j==0):
    totallywrong.append((m,i))
  m = m+1

In [ ]:
for i in almostright[0:10]:
  print(dataarr_test[i[0]],i[1])


In [ ]:
for i in totallywrong[0:10]:
  print(dataarr_test[i[0]],i[1])

In [ ]:
print(metrics)

In [ ]:
t = []
c1 = []
c2 = []
for i in metrics:
  t.append(i[0])
  c1.append(i[1])
  c2.append(i[2])

In [ ]:
%matplotlib inline               
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.plot(t)
plt.plot(c1)
plt.plot(c2)                # plot the points in the array a
plt.title("My first figure")  # Add a title to the figure
plt.xlabel("Models")    # Give a label to the x-axis
plt.ylabel("Accuracy means"); 